In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
pip install transformers datasets evaluate scikit-learn

In [ ]:
pip install -U datasets huggingface_hub fsspec


In [ ]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

In [ ]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sent_train.csv: 0.00B [00:00, ?B/s]

sent_valid.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/9543 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
label2id = {"Bearish": 0, "Bullish": 1, "Neutral": 2}
id2label = {v: k for k, v in label2id.items()}

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Tokenization
def preprocess(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

encoded_ds = dataset.map(preprocess, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/9543 [00:00<?, ? examples/s]

Map:   0%|          | 0/2388 [00:00<?, ? examples/s]

In [ ]:
encoded_ds.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"])


In [ ]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)

# Metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-finance-sentiment",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-10-3380617199.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()
results = trainer.evaluate()
print("Final Evaluation:", results)

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.400700,0.337274,0.870603,0.831690
2,0.217600,0.405050,0.881491,0.841353
3,0.075300,0.525775,0.888191,0.855016


Final Evaluation: {'eval_loss': 0.5257754921913147, 'eval_accuracy': 0.8881909547738693, 'eval_f1_macro': 0.8550159545011877, 'eval_runtime': 69.7766, 'eval_samples_per_second': 34.223, 'eval_steps_per_second': 2.15, 'epoch': 3.0}


In [ ]:
import torch

model.eval()
def predict_sentiment(texts):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

    return [id2label[p.item()] for p in preds]


example_texts = [
    "Stock prices are expected to rise after the earnings report.",
    "The market crashed hard today.",
    "Nothing major happened in the financial sector this week."
]

predictions = predict_sentiment(example_texts)

for text, label in zip(example_texts, predictions):
    print(f"Text: {text}\nPredicted Sentiment: {label}\n")

Text: Stock prices are expected to rise after the earnings report.
Predicted Sentiment: Bullish

Text: The market crashed hard today.
Predicted Sentiment: Bearish

Text: Nothing major happened in the financial sector this week.
Predicted Sentiment: Neutral

